# BTSnoop Log Sniffer

This notebook demonstrates the process of parsing bluetooth packet data from android hci logs.

In [28]:
!pip install fpdf
!pip install kaleido
!pip install dataframe_image

  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 17.3 MB/s eta 0:00:0000:0100:01


In [ ]:
import fpdf
from fpdf import FPDF
import dataframe_image as dfi
import matplotlib.pyplot as plt
import time 

import binascii
import btsnoop.bt.hci as hci
import btsnoop.btsnoop.btsnoop as bts
import btsnoop.bt.hci_uart as hci_uart
import btsnoop.bt.hci_cmd as hci_cmd
import btsnoop.bt.hci_evt as hci_evt
import btsnoop.bt.hci_acl as hci_acl
import btsnoop.bt.hci_sco as hci_sco
import btsnoop.bt.l2cap as l2cap
import btsnoop.bt.att as att
import btsnoop.bt.smp as smp
import pandas as pd
from pprint import pprint

pd.set_option('display.max_columns', None)

records = bts.parse("./bt_logfiles/btsnoop_hci_10022024_1930.log")

records[0]

# TODO: SQLite storage for known devices

btsnoop capture file version 1, type 1002


(1,
 47,
 3,
 datetime.datetime(2024, 10, 2, 17, 34, 2, 987322),
 b'\x04>,\r\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\xff\x7f\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x12\x11\tJBL VIBE BUDS-LE')

In [14]:
rows = []
for record in records:
    row = {}
    seq = record[0]
    packet_length = record[1]
    flags_raw = record[2]
    flags = bts.flags_to_str(flags_raw)
    pkt_src = flags[0].capitalize()
    pkt_dst = flags[1].capitalize()
    pkt_type = flags[2]
    time = record[3].strftime("%m-%d-%Y %H:%M:%S.%f")
    pkt_data = record[4]

    direction = bts.flags_to_direction(flags_raw)

    hci_pkt_type, hci_pkt_data = hci_uart.parse(record[4])
    hci_type = hci_uart.type_to_str(hci_pkt_type)

    row.update({'sequence_number': seq,
         'packet_length': packet_length,
         'time_rcvd': time,
         'packet_source': pkt_src,
         'packet_destination': pkt_dst,
         'packet_type': pkt_type,
         'hci_packet_type': hci_type,
         'hci_packet_code': hci_pkt_type,
         'packet_data_unparsed': pkt_data,
         })

    if hci_pkt_type == hci_uart.HCI_CMD:
        opcode, length, data = hci_cmd.parse(hci_pkt_data)
        cmd_evt_l2cap = hci_cmd.cmd_to_str(opcode)
        row.update({'opcode':opcode,
                    'hci_cmd_data':data,
                    'cmd_evt_l2cap':cmd_evt_l2cap})
    elif hci_pkt_type == hci_uart.HCI_EVT:
        hci_data = hci_evt.parse(hci_pkt_data)
        evtcode, data = hci_data[0], hci_data[-1]
        cmd_evt_l2cap = hci_evt.evt_to_str(evtcode)
        row.update({'event_code':evtcode,
                    'HCI_EVT_data':data,
                    'cmd_evt_l2cap':cmd_evt_l2cap})
    elif hci_pkt_type == hci_uart.SCO_DATA:
        handle, ps, length, data = hci_sco.parse(hci_pkt_data)

        l2cap_length, l2cap_cid, l2cap_data = l2cap.parse(hci_data[2], data)
        data = binascii.hexlify(data)
        data = len(data) > 30 and data[:30] + "..." or data

        row.update({'handle':handle,
                    'ps':ps,
                    'length':length,
                    'data':data})
        # print(handle, ps, length, data)
        raise Exception('DEBUG: SCO Data!')
    elif hci_pkt_type == hci_uart.ACL_DATA:
        hci_data = hci_acl.parse(hci_pkt_data)
        l2cap_length, l2cap_cid, l2cap_data = l2cap.parse(hci_data[2], hci_data[4])
        row.update({'l2cap_length':l2cap_length,
                    'l2cap_cid':l2cap_cid,
                    'l2cap_data':l2cap_data})

        if l2cap_cid == l2cap.L2CAP_CID_LE_ATT:
            att_opcode, att_data = att.parse(l2cap_data)
            cmd_evt_l2cap = att.opcode_to_str(att_opcode, att_data)
            data = att_data
            row.update({'att_opcode':att_opcode,
                        'cmd_evt_l2cap':cmd_evt_l2cap,
                        'data':data})

        elif l2cap_cid == l2cap.L2CAP_CID_LE_SMP:
            smp_code, smp_data = smp.parse(l2cap_data)
            cmd_evt_l2cap = smp.code_to_str(smp_code)
            data = smp_data
            row.update({'smp_code':smp_code,
                        'cmd_evt_l2cap':cmd_evt_l2cap,
                        'data':data})

        elif l2cap_cid == l2cap.L2CAP_CID_LE_SCH:
            sch_code, sch_id, sch_length, sch_data = l2cap.parse_sch(l2cap_data)
            cmd_evt_l2cap = l2cap.sch_code_to_str(sch_code)
            data = sch_data
            row.update({'sch_code':sch_code,
                        'sch_id':sch_id,
                        'sch_length':sch_length,
                        'sch_data':sch_data,
                        'cmd_evt_l2cap':cmd_evt_l2cap,
                        'data':data})
    else:
        raise Exception('Unknown HCI Packet Type')

    rows.append(row)

    

[source](https://github.com/nccgroup/BLE-Replay/blob/master/btsnoop/btsnoop/bt/hci_evt.py)

The HCI LE Meta Event is used to encapsulate all LE Controller specific events.
The Event Code of all LE Meta Events shall be 0x3E. The Subevent_Code is
the first octet of the event parameters. The Subevent_Code shall be set to one
of the valid Subevent_Codes from an LE specific event


HCI inherently cannot differentiate between packet types. Hence a common physical interface is used with the indicators that are sent right before the packet is sent. These indicators are as follows

| HCI Packet Type | HCI Packet Indicator |
| --------------- | -------------------- |
| HCI Command Packet | 0x01 |
| HCI ACL Data Packet | 0x02 |
| HCI Synchronous Data Packet | 0x03 |
| HCI Event Packet | 0x04 |
| HCI ISO Data Packet | 0x05 |

LE Meta events encapsulate all LE Controller events and have a code of 0x03

In [15]:
print(rows[0])

{'sequence_number': 1, 'packet_length': 47, 'time_rcvd': '10-02-2024 17:34:02.987322', 'packet_source': 'Controller', 'packet_destination': 'Host', 'packet_type': 'event', 'hci_packet_type': 'HCI_EVT', 'hci_packet_code': 4, 'packet_data_unparsed': b'\x04>,\r\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\xff\x7f\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x12\x11\tJBL VIBE BUDS-LE', 'event_code': 62, 'HCI_EVT_data': b'\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\xff\x7f\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x12\x11\tJBL VIBE BUDS-LE', 'cmd_evt_l2cap': 'EVENT LE_Meta_Event (0x3e)'}


[Reference](https://datatracker.ietf.org/doc/html/rfc1761)

Android bluetooth logs come in the **Snoop Version 1 Packet Capture File Format** which is similar to the second version developed by Sun Microsystems in 1995. When we capture logs of this format, we obtain arrays of octets (8 bit packets of information), with each array item corresponding to a packet record.


In [16]:
ble_data = pd.DataFrame(rows)
ble_data.columns

Index(['sequence_number', 'packet_length', 'time_rcvd', 'packet_source',
       'packet_destination', 'packet_type', 'hci_packet_type',
       'hci_packet_code', 'packet_data_unparsed', 'event_code', 'HCI_EVT_data',
       'cmd_evt_l2cap', 'l2cap_length', 'l2cap_cid', 'l2cap_data', 'opcode',
       'hci_cmd_data', 'att_opcode', 'data', 'sch_code', 'sch_id',
       'sch_length', 'sch_data'],
      dtype='object')

In [17]:
ble_data[ble_data['hci_packet_type'] == 'HCI_EVT'][['sequence_number', 'packet_length', 'time_rcvd', 'packet_source', 'packet_destination', 'packet_type', 'hci_packet_code', 'event_code', 'packet_data_unparsed', 'HCI_EVT_data', 'cmd_evt_l2cap']].head()

,sequence_number,packet_length,time_rcvd,packet_source,packet_destination,packet_type,hci_packet_code,event_code,packet_data_unparsed,HCI_EVT_data,cmd_evt_l2cap
0,1,47,10-02-2024 17:34:02.987322,Controller,Host,event,4,62.0,"b'\x04>,\r\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x...",b'\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\x...,EVENT LE_Meta_Event (0x3e)
1,2,57,10-02-2024 17:34:03.149816,Controller,Host,event,4,62.0,"b""\x04>6\r\x01\x13\x00\x01\xcb9\x19\xdd\xe3l\x...","b""\x01\x13\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\x...",EVENT LE_Meta_Event (0x3e)
2,3,47,10-02-2024 17:34:03.151213,Controller,Host,event,4,62.0,"b'\x04>,\r\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x...",b'\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\x...,EVENT LE_Meta_Event (0x3e)
3,4,57,10-02-2024 17:34:03.311777,Controller,Host,event,4,62.0,"b""\x04>6\r\x01\x13\x00\x01\xcb9\x19\xdd\xe3l\x...","b""\x01\x13\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\x...",EVENT LE_Meta_Event (0x3e)
4,5,47,10-02-2024 17:34:03.313999,Controller,Host,event,4,62.0,"b'\x04>,\r\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x...",b'\x01\x1b\x00\x01\xcb9\x19\xdd\xe3l\x01\x00\x...,EVENT LE_Meta_Event (0x3e)


In [18]:
ble_data[ble_data['hci_packet_type'] == 'HCI_EVT'].shape

(22642, 23)

In [19]:
ble_data[ble_data['hci_packet_type'] == 'HCI_EVT']['att_opcode'].isna().all()

True

In [24]:
ble_data['hci_packet_type'].unique()

array(['HCI_EVT', 'ACL_DATA', 'HCI_CMD'], dtype=object)

In [26]:
# att_data = ble_data[(ble_data['hci_packet_type'] == 'ACL_DATA') & (ble_data['l2cap_cid'] == l2cap.L2CAP_CID_LE_ATT)].dropna(axis=1, how='all')
sco_data = ble_data[(ble_data['hci_packet_type'] == 'ACL_DATA')].dropna(axis=1, how='all')
sco_data.head()

,sequence_number,packet_length,time_rcvd,packet_source,packet_destination,packet_type,hci_packet_type,hci_packet_code,packet_data_unparsed,cmd_evt_l2cap,l2cap_length,l2cap_cid,l2cap_data,att_opcode,data,sch_code,sch_id,sch_length,sch_data
55,56,14,10-02-2024 17:34:09.550175,Controller,Host,data,ACL_DATA,2,b'\x02\x0b \t\x00\x05\x00E\x00\xd0\r\x08\x08\xfc',NaN,5.0,69.0,b'\xd0\r\x08\x08\xfc',NaN,NaN,NaN,NaN,NaN,NaN
56,57,11,10-02-2024 17:34:09.552428,Host,Controller,data,ACL_DATA,2,b'\x02\x0b\x00\x06\x00\x02\x00A\x00\xd2\r',NaN,2.0,65.0,b'\xd2\r',NaN,NaN,NaN,NaN,NaN,NaN
79,80,14,10-02-2024 17:34:11.894999,Controller,Host,data,ACL_DATA,2,b'\x02\x0b \t\x00\x05\x00E\x00\xe0\r\x08\t\xe2',NaN,5.0,69.0,b'\xe0\r\x08\t\xe2',NaN,NaN,NaN,NaN,NaN,NaN
80,81,11,10-02-2024 17:34:11.897429,Host,Controller,data,ACL_DATA,2,b'\x02\x0b\x00\x06\x00\x02\x00A\x00\xe2\r',NaN,2.0,65.0,b'\xe2\r',NaN,NaN,NaN,NaN,NaN,NaN
209,210,34,10-02-2024 17:34:32.592301,Controller,Host,data,ACL_DATA,2,b'\x02\x0b \x1d\x00\x19\x00A\x00\x99\xef+AT+IP...,NaN,25.0,65.0,"b'\x99\xef+AT+IPHONEACCEV=1,1,6\r\xfd'",NaN,NaN,NaN,NaN,NaN,NaN
